In [1]:
import openai
import csv
import os
import openai
import pandas as pd
import collections
import ast
import time
import collections
import json
import sqlite3
import tiktoken
import xxhash
import numpy as np
from openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

In [2]:
# fill in the openai api key
os.environ["OPENAI_API_KEY"] = ""

openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
import pickle 
def save(fname, d):
    with open(fname, 'wb') as f:
        pickle.dump(d, f)


In [4]:
import time as time
from multiprocessing import Process, Queue
import query_module

# def handler(signum, frame):
#     print('time out')
#     raise TimeoutError("Query execution exceeded the allowed time limit.")


def evalfunc(sql_source, sql_target, database, source='kaggle'):
    assert source in ['kaggle', 'bird']
    db_path = ''
    if source == 'kaggle':
        db_path = f'./databases/{database}/{database}.sqlite'
    elif source == 'bird':
        db_path = f'./bird_dev/dev_databases/{database}/{database}.sqlite'
    if not os.path.isfile(db_path):
        print("cannot find file", db_path)
        return False
    timeout = 120
    output = Queue()
    query_process = Process(target=query_module.execute_query, args=(db_path, sql_source, output))
    query_process.start()
    output_hash = ''
    
    try:
        # Connect to sqlite db
        # Execute both!
        source_results = None
        source_results = output.get(True, timeout+5)
        query_process.join(timeout)
        if query_process.is_alive():
            print("process terminated")
            query_process.terminate()  # Terminate the process
            query_process.join()  # Make sure it's cleaned up
            return False, [Exception('SQL query took too much time to execute.')]
        if isinstance(source_results, Exception):
            raise source_results
        output_hash = xxhash.xxh128_hexdigest(str(len(source_results)), seed=123)
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()
        target_results = cursor.execute(sql_target).fetchall()
        cursor.close()
        connection.close()
        # If the lengths don't match... there's no hope
        if len(source_results) != len(target_results):
            # (result matches or not, valid, hash)
            return False, []
        if 'ORDER BY' in sql_target:
            for a, b in zip(source_results, target_results):
                # NOTE: we are doing compares that are column-order independent
                # hence the sorting and the weird key (since we may have mixed
                # types in a row)
                lhs = tuple(sorted(list(a), key=lambda x: hash(x)))
                rhs = tuple(sorted(list(b), key=lambda x: hash(x)))
                output_hash = xxhash.xxh128_hexdigest(output_hash + str(lhs), seed=123)
                if lhs != rhs:
                    # Oh no, a row doesn't match!
                    return False, []
        else:
            lset, rset = set(), set()
            for a, b in zip(source_results, target_results):
                # NOTE: we are doing compares that are column-order independent
                # hence the sorting and the weird key (since we may have mixed
                # types in a row)
                lset.add(tuple(sorted(list(a), key=lambda x: hash(x))))
                rset.add(tuple(sorted(list(b), key=lambda x: hash(x))))
            output_hash = xxhash.xxh128_hexdigest(str(lset), seed=123)
            if lset != rset:
                # Oh no, rows don't match!
                return False, []
    # If we hit an error, that's not a match I guess...
    except Exception as ex:
        print(ex)
        return False, [ex]
    return True, []


def outputHash(sql_source, database):
    db_path = f'./databases/{database}/{database}.sqlite'
    output_hash = ''
    try:
        # Connect to sqlite db
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()
        source_results = cursor.execute(sql_source).fetchall()
        output_hash = xxhash.xxh128_hexdigest(str(len(source_results)), seed=123)
        if 'ORDER BY' in sql_source:
            for a in source_results:
                lhs = tuple(sorted(list(a), key=lambda x: hash(x)))
                output_hash = xxhash.xxh128_hexdigest(output_hash + str(lhs), seed=123)
        else:
            lset = set()
            for a in source_results:
                lset.add(tuple(sorted(list(a), key=lambda x: hash(x))))
            output_hash = xxhash.xxh128_hexdigest(str(lset), seed=123)
    except Exception as ex:
        return False
    finally:
        cursor.close()
        connection.close()
    return output_hash


def execute(sql, database, source):
    assert source in ['kaggle', 'bird']
    db_path = ''
    if source == 'kaggle':
        db_path = f'./databases/{database}/{database}.sqlite'
    elif source == 'bird':
        db_path = f'./bird_dev/dev_databases/{database}/{database}.sqlite'
        
    if not os.path.isfile(db_path):
        print("cannot find file")
        return False
    results = ''
    try:
        # Connect to sqlite db
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()
        results = cursor.execute(sql).fetchall()
    # If we hit an error, that's not a match I guess...
    except KeyboardInterrupt:
        cursor.close()
        connection.close()
        print("KeyboardInterrupt")
        return False
    except Exception as ex:
        cursor.close()
        connection.close()
        print(ex)
        return False
    finally:
        cursor.close()
        connection.close()
    return results

In [49]:
def GPT4_turbo_generation(prompt, t = 0.0):
    response = client.chat.completions.create(
        model = 'gpt-4-turbo',
        messages=[{"role": "user", "content": prompt}],
        n = 1,
        stream = False,
        temperature=t,
        max_tokens=4096,
        logprobs=True,
    )
    logprobs = [token.logprob for token in response.choices[0].logprobs.content]
    perplexity_score = np.exp(-np.mean(logprobs))
    return response.choices[0].message.content.strip(), perplexity_score

def GPT4o_generation(prompt, t = 0.0):
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages=[{"role": "user", "content": prompt}],
        n = 1,
        stream = False,
        temperature=t,
        max_tokens=4096,
        logprobs=True,
    )
    logprobs = [token.logprob for token in response.choices[0].logprobs.content]
    perplexity_score = np.exp(-np.mean(logprobs))
    return response.choices[0].message.content.strip(), perplexity_score



def GPT35_generation(prompt, t = 0.0):
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages=[{"role": "user", "content": prompt}],
        n = 1,
        stream = False,
        temperature=t,
        max_tokens=4096,
        logprobs=True,
    )
    logprobs = [token.logprob for token in response.choices[0].logprobs.content]
    perplexity_score = np.exp(-np.mean(logprobs))
    return response.choices[0].message.content.strip(), perplexity_score

In [6]:
df = pd.read_csv('kaggle_dataset.csv')

In [7]:
bird_data_df = pd.read_json('bird_dev.json')
bird_data_df.reset_index(level=0, inplace=True)

In [8]:
assert len(bird_data_df) == 1534

In [9]:
userstudy = []
with open('./logs/user_study.pkl', 'rb') as f:
    userstudy = pickle.load(f)

In [10]:
survey_questions = []
# add original gold query inside
for d in userstudy:
    if 'Question2Ask' in d:
        assert len(d['Question2Ask']) == len(d['Answer2Question']), print(d['nl'])
    q = d['nl']
    sql = df.loc[df['nl'] == q]['sql'].values
    d['gold'] = sql[0]
    d["target_schema"] = df.loc[df['nl'] == q]['target_schema'].values
    survey_questions.append(q)

print(len(df))
# drop the user study questions
df = df[~df['nl'].isin(survey_questions)]
print(len(df))
    

272
208


In [11]:
df = df.reset_index()

In [148]:
def clean_query(sql_query):
    sql_query = sql_query.replace("```sql", '')
    sql_query = sql_query.replace("```", '')
    sql_query = sql_query.replace(';', '')
    sql_query = sql_query.replace('"""', '')
    if 'SELECT' not in sql_query.upper()[:10]:
        sql_query = 'SELECT ' + sql_query
    return sql_query


In [15]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


In [16]:
feedback_prefix_v1='''/* some examples are provided */
/* example question: */
How many acres burned in fires in California each year between 2000 and 2005?
/* example gold sql query*/
SELECT\n  SUM(FIRE_SIZE),\n  FIRE_YEAR\nFROM Fires\nWHERE\n  State = "CA" AND FIRE_YEAR BETWEEN 2000 AND 2005\nGROUP BY\n  FIRE_YEAR
/* example clarification question*/
What information should the output table contain? a) two columns: the total acres burned and the year, b) one column: the total acres burned for each year, c) one column: the total acres burned across all target years, d) other (please specify).
/* example reasoning */
Output table is determined by the SELECT clause in the gold sql query. The gold query uses ‘SELECT  SUM(FIRE_SIZE), FIRE_YEAR’. As a result, the output table has two columns, the total acres burned and the year. Hence, choice a is correct.
/* example  answer*/
answer_to_cq = "a) two columns: the total acres burned and the year"

/* example question: */
Which states had the largest number of fires in 2001?
/* example gold sql query*/
SELECT\n  State\nFROM Fires\nWHERE\n  FIRE_YEAR = 2001\nGROUP BY\n  State\nORDER BY\n  COUNT(*) DESC\nLIMIT 1;
/* example clarification question*/
Is the largest number of fires referring to? a) the total size of all fire incidents, b) the number of fire incidents, c) the largest size of all fire incidents, d) other (please specify).
/* example reasoning */
The clarification question is asking about how to represent the largest number of fires. The gold query uses ‘ORDER BY COUNT(*) DESC LIMIT 1’ to find the largest number of fires. As a result, choice a is correct.
/* example  answer*/
answer_to_cq = "b) the number of fire incidents"

/* example question: */
What was the most common cause of fire between 2000 and 2005?
/* example gold sql query*/
SELECT\n  STAT_CAUSE_DESCR\nFROM Fires\nWHERE\n  FIRE_YEAR BETWEEN 2000 AND 2005\nGROUP BY\n  STAT_CAUSE_DESCR\nORDER BY\n  COUNT(*) DESC\nLIMIT 1;
/* example clarification question*/
Which information should be used to represent the 'cause of fire'? a) the code that represents the cause, b) the description of the cause, c) both the code and the description of the cause, d) other (please specify).
/* example reasoning */
The clarification question is asking for which column should be used to represent the cause of fire. The gold query uses the STAT_CAUSE_DESCR to represent the cause. As a result, choice b is correct.
/* example  answer*/
answer_to_cq = "b) the description of the cause"

/* example question: */
Whose CDs sells best?
/* example gold sql query*/
SELECT\n  artist\nFROM torrents\nGROUP BY\n  artist\nORDER BY\n  SUM(totalSnatched) DESC\nLIMIT 1;
/* example clarification question*/
Which column should be used to identify music related to 'CD'? a) groupName, b) tag, c) releaseType, d) other (please specify)
/* example reasoning */
The gold query does not use a WHERE clause to filter the CDs. Hence, the CD information is not contained in the tag column or the release type column. As a result, choice a, b, and c are all wrong.
/* example  answer*/
answer_to_cq = “d) Consider all music; No filter on ‘CD’ ”

/* example question: */
How many people wrote comments for the question "Any additional notes or comments."? */
/* example gold sql query*/
SELECT COUNT(T1.UserID) FROM Answer AS T1 INNER JOIN Question AS T2 ON T1.QuestionID = T2.questionid WHERE T2.questiontext LIKE 'Any additional notes or comments' AND T1.AnswerText IS NOT NULL
/* example clarification question*/
How to determine if a user has provided comments? a) no check needed, b) see if `AnswerText` column has empty string, c) other (please specify).
/* example reasoning */
In the gold SQL query, it checks “T1.AnswerText IS NOT NULL”. Hence, choice a and b are both wrong.
/* example  answer*/
answer_to_cq = "c) ‘wrote comments’ imply `AnswerText` is not a NULL value".

/* example question: */
Calculate the difference between the number of customers and the number of subscribers who did the trip in June 2013. 
/* example gold sql query*/
SELECT SUM(IIF(subscription_type = 'Subscriber', 1, 0)) - SUM(IIF(subscription_type = 'Customer', 1, 0)) FROM trip WHERE start_date LIKE '6/%/2013%'
/* example clarification question*/
What predicate value should be used to determine a trip in June 2013? a) start_data > 06/2013, b) start_data = ‘June 2013’, c) other (please specify).
/* example reasoning */
The gold sql query uses start_date LIKE '6/%/2013%' to find trips in June 2013.
/* example  answer*/
answer_to_cq = "c) start_date LIKE '6/%/2013%'"


/* example question: */
Identify the players who weigh 120 kg.
/* example gold sql query*/
SELECT T2.PlayerName FROM weight_info AS T1 INNER JOIN PlayerInfo AS T2 ON T1.weight_id = T2.weight WHERE T1.weight_in_kg = 120
/* example clarification question*/
What fields should be contained in the output? a) one column of player name, b) one column of player id, c) two columns of player name and player ids, d) other (please specify).
/* example reasoning */
The gold query selects ‘SELECT T2.PlayerName’. Hence, a is correct.
/* example  answer*/
answer_to_cq = "a) one column of player name"

/* example question: */
How many reviews are created for the podcast "Scaling Global" under?
/* example gold sql query*/
SELECT COUNT(T2.content) FROM podcasts AS T1 INNER JOIN reviews AS T2 ON T2.podcast_id = T1.podcast_id WHERE T1.title = 'Scaling Global'
/* example clarification question*/
Which column represents the reviews? a) `podcast` column, b) `content` column, c) other (please specify).
/* example reasoning */
The gold query uses “COUNT(T2.content)” to determine the number of reviews. Hence, b is correct in which the `content` column represents the reviews.
/* example  answer*/
answer_to_cq = "b) `content` column"
\n\n
'''

# feedback_v2 = """Your task is to answer the multiple choice clarification question truthfully based on the Gold Query.

# Natural Language Question: ```{nlq}```
# Gold Query: ```{query}```
# The gold query is the sql answer to the natural language question

# Multiple Choice Clarification Question: ```{question}```

# Answer the above clarification question truthfully based on the Gold Query.

# First, reason which portion of the gold query answers the clarification question.
# Next, consider the correctness of each multiple choice answers based only on the gold query.
# Lastly, output the answer with format: answer_to_cq = "".
# If none of the multiple choices are correct or you selected `other (please specify)`, give a short answer to the clarification question after `other (please specify)`.


# Let's think step by step:
# """


feedback_v2 = """/* Given the following Natural Language Question: */
{nlq}
/* And the following Gold Query: */
{query}
/* Answer the following multiple choice clarification question truthfully based on the Gold Query: */
{question}

/* Follow these steps:
1. Identify which portion of the Gold Query answers the clarification question.
2. Evaluate the correctness of each multiple choice answer based only on the Gold Query.
3. If none of the choices are correct or you select "other (please specify)", provide a short answer for the clarification question.
4. Output the final answer in the format: answer_to_cq = "".

Let’s proceed step by step. */
"""



In [54]:
cq_prefix_v1 = '''/* some examples are provided */
/* example question: */
Which artist/group is most productive?
/* example previous clarification questions and user replies: */
clarification questions: How to rank artist/group productivity? a) rank by the number of records produced, b) rank by the total number of downloads, c) other (please specify).
user: b) rank by the total number of downloads```
/* example reasoning and remaining ambiguity type*/
It is clear that the SQL answer should use ORDER BY and LIMIT 1 based on the sum of total downloads. However, it is unclear what columns should be used to represent the 'artist/group'.  Both the `artist` and the `groupName` columns contain information about 'artist/group'. ’‘AmbTableColumn’ remains.
/* example clarification question */
mul_choice_cq = "Which columns represent the 'artist/group' information? a) the artist column only, b) the groupName column only, c) both the artist column and the groupName column, d) other (please specify).”```

/* example question: */
Which Premier League matches ended in a draw in 2016?
/* example previous clarification questions and user replies: */
clarification questions: Is the year '2016' referring to? a) season is 2016, b) season is either 2015/2016 or 2016/2017, c) the date time is at year 2016, d) other (specify).
user: a) season is 2016,
clarification questions: How to find the 'Premier league'? a) consider all leagues, b) consider only the league with name 'Premier League', c) other (specify).
user: b) consider only the league with name 'Premier League'
/* example reasoning and remaining ambiguity type*/
It is clear that the SQL answer to this question needs to contain a WHERE clause for three conditions: 'Premier League', 'draw', and 'in 2016'. However, the question did not specify what fields should be contained in the output table. 'AmbOutput' remains.
/* example clarification question */
mul_choice_cq = “What fields represent the target 'matches'? a) all fields from football data table, b) the `league` column, c) other (specify).”

/* example question: */
Which type of crime has the highest rate of ‘Investigation complete’?
/* example previous clarification questions and user replies: */
No previous clarification questions.
/* example reasoning and remaining ambiguity type*/
It is clear that the SQL answer to this question needs to contain a WHERE clause to find crimes that have 'Investigation complete' outcomes, uses ORDER BY and LIMIT 1 to find the type of crime with the highest rate, and the output table has only one row. However, it needs to be clarified i) what predicate value should be used for 'Investigation complete', and ii) how to represent the 'rate', and iii) if the output table contains only the crime type column or the crime type column with the highest rate aggregate. Hence, this question is ambiguous because of 'AmbVal', 'AmbQuestion', and ‘AmbOutput’.
/* example clarification question */
mul_choice_cq = “What information should be used to find 'Investigation complete'? a) see if outcome contains the phrase 'Investigation complete', b)  see if outcome is 'Investigation complete; no suspect identified', c) other (please specify).”

/* example question: */
For award winners, which position has the most hall of fame players?
/* example previous clarification questions and user replies: */
clarification questions: How should the 'position' for players be identified? a) by the `award_id` column, b) by the `category` column, c) other (please specify).
user: c)  by the `note` column
/* example reasoning and remaining ambiguity type*/
It is clear that the answer should use the `note` column for player ‘positions’. However, it is unclear what fields should contain in the output table. Hence ‘AmbOutput’ remains.
/* example clarification question */
mul_choice_cq = “What fields should be contained in the output table? a) one field: the position, b) two fields: the position and the number of hall-of-fame players, c) other (please specify).”

/* example question: */
How many Wisconsin school districts receive federal funding?
/* example previous clarification questions and user replies: */
clarification question: How to determine if a district has received federal funding? a) based on the t_fed_rev is larger than 0, b) the answer does not need to consider this aspect, c) other (please specify).
user: c) every school in `FINREV_FED_17` table has received federal funding.
/* example reasoning and remaining ambiguity type*/
It is clear that every school in `FINREV_FED_17` table have received federal funding. However, it is unclear if the word ‘Wisconsin’ refers to the state or the school district. Hence, ‘AmbQuestion’ remains.
/* example clarification question */
mul_choice_cq = “Is 'Wisconsin school districts' referring to? a) all school districts in the state Wisconsin, b) school districts with names that contain Wisconsin, c) other (please specify).”

/* example question: */
How many 2-year public schools are there in "California"?
/* example previous clarification questions and user replies: */
clarification question: Which column(s) should be used to find ‘2-year public schools’? a) `level` column, b) `control` column, c) other (please specify).
user: c) use both `level` and `control` columns to find ‘2-year public schools’ information.
/* example reasoning and remaining ambiguity type*/
It is clear that the correct SQL answer should have a WHERE clause with filters based on the `level` and `control` columns. However, it is unclear what predicate values should be used for these two columns. Hence, ‘AmbValue’ remains.
/* example clarification question */
mul_choice_cq = “What predicate values should be used for the `level` and `control` columns to find  ‘2-year public schools’? a) ‘2-year’ and ‘public’ b) ‘2’ and ‘public, c) other (please specify)’.” 

/* example question: */
Calculate the total beat of the crimes reported in a community area in the central side with a population of 50,000 and above.
/* example previous clarification questions and user replies: */
clarification question: What column and predicate value should be used to determine ‘central side’? a) Column `side` in table `Community_Area` with value ‘central’, b) Column `side` in table `Community_Area` with value ‘Central’, c) other (please specify).
user: b) Column `side` in table `Community_Area` with value ‘Central’
/* example reasoning and remaining ambiguity type*/
It is clear that the output table should contain a single number and use the predicate ‘Central’ in `Community_Area`.`side`; However, it is not clear which column of statistics is ‘total beat’ referring to. Hence, AmbTableColumn remains.
/* example clarification question */
mul_choice_cq = “Which column is related to ‘total beat’? a) `Crime`.`beat`, b) `Crime`.`report_no`, c) other (please specify).”

/* example question: */
Of all the nonessential genes that are not of the motorprotein class and whose phenotype is cell cycle defects, how many do not have a physical type of interaction?
/* example previous clarification questions and user replies: */
No previous clarification questions.
/* example reasoning and remaining ambiguity type*/
It is clear that ‘phenotype’ is referring to the `Phenotype` column, ‘motorprotein class’ is referring to the `class` column, ‘nonessential genes’ is referring to the `essential` column, and `physical type` is referring to the `type` column. However, it is unclear what fields should be contained in the output table, and hence ‘AmbOutput’ remains.
/* example clarification question */
mul_choice_cq = “What fields should be included in the output table? a) One column for the number of genes b) Two columns for GeneID and physical type c) Other (please specify).”
\n\n
'''

# SRA = """Your task is to ask the user a single multiple choice clarification question to help you find the correct SQL query.
# User does not understand SQL. The clarification question should be comprehensive by people with no coding experience.

# QUESTION: ```{question}```
# DATABASE SCHEMA:```{schema}```

# Previous Clarification Questions and User Feedback: ```{cqs}```

# List of Incorrect Queries: ```{sqls}```
# The above SQL Queries are wrong.

# Here are four ambiguity categories to consider:
#     - AmbQuestion: Is the question itself ambiguous?
#     - AmbTableColumn: Is there ambiguity in mapping the entities from QUESTION to tables and columns in DATABASE SCHEMA?
#     - AmbOutput: What fields and how many fields should be contained in the output table?
#     - AmbValue: What predicate value should be used to filter results?

# Let's think step by step.

# STEP 1, Summarize the information that is clear based on the ansewrs to previous clarification questions and incorrect queries.

# STEP 2, Evaluate whether AmbQuestion, AmbColumn, AmbOutput, and AmbValue remain in formulating a SQL query to answer the QUESTION, considering each category individually.

# STEP 3, Ask a multiple-choice clarification question to clarify the remaining ambiguities and help you find the correct SQL query. Ensure it's simple enough for someone without coding experience to understand.
#         Use format: mul_choice_cq = "".
        
# Answer:
# """

# # decide when to stop
# SRA_ES = """Your task is to identify the remaining ambiguity.
# If there are remaining ambiguities, then ask the user a single multiple choice clarification question to help you find the correct SQL query.
# User does not understand SQL. The clarification question should be comprehensive by people with no coding experience.

# QUESTION: ```{question}```
# DATABASE SCHEMA:```{schema}```

# Previous Clarification Questions and User Feedback: ```{cqs}```

# List of Incorrect Queries: ```{sqls}```
# The above SQL Queries are wrong.

# Here are four ambiguity categories to consider:
#     - AmbQuestion: Is the question itself ambiguous?
#     - AmbTableColumn: Is there ambiguity in mapping the entities from QUESTION to tables and columns in DATABASE SCHEMA?
#     - AmbOutput: What fields and how many fields should be contained in the output table?
#     - AmbValue: What predicate value should be used to filter results?

# Let's think step by step.

# STEP 1, Summarize the information that is clear based on the ansewrs to previous clarification questions.

# STEP 2, Evaluate whether AmbQuestion, AmbColumn, AmbOutput, and AmbValue remain in formulating a SQL query to correctly answer the QUESTION, considering each category individually.

# STEP 3, If no remaining ambiguities are identified, then output "NO AMBIGUITY".
#         Otherwise, ask a multiple-choice clarification question to clarify the remaining ambiguities and help you find the correct SQL query. Ensure it's simple enough for someone without coding experience to understand. Use format: mul_choice_cq = "".
        
# Answer:
# """

SRA = """/* Ask the user a new multiple choice clarification question to help you find the correct SQL answer for the following question: */
{question}
/* Given the following database schema: */
{schema}
/* And the following incorrect sql answers: */
{sqls}
/* And the following previous clarification questions and user replies: */
{cqs}

/* Consider the following ambiguity categories:
    - AmbQuestion: Is the question itself ambiguous?
    - AmbTableColumn: Is there ambiguity in mapping the entities from the QUESTION to tables and columns in the DATABASE SCHEMA?
    - AmbOutput: What fields and how many fields should be included in the output table?
    - AmbValue: What predicate value should be used to filter results?
*/

/* The clarification question should be easy to understand for people with no coding experience. */

/* Let's think step by step to generate the helpful multiple choice clarification question.
1. Summarize the clear information based on previous clarification questions and incorrect queries.
2. Evaluate whether AmbQuestion, AmbTableColumn, AmbOutput, and AmbValue remain in formulating an SQL query, considering each category individually.
3. Ask a new multiple-choice question to address the remaining ambiguities and assist in identifying the correct SQL query. Use format: mul_choice_cq = "".
*/

"""


SRA_ES = """/* Ask the user a new multiple choice clarification question to help you find the correct SQL answer for the following question: */
{question}
/* Given the following database schema: */
{schema}
/* And the following incorrect sql answers: */
{sqls}
/* And the following previous clarification questions and user replies: */
{cqs}

/* Consider the following ambiguity categories:
    - AmbQuestion: Is the question itself ambiguous?
    - AmbTableColumn: Is there ambiguity in mapping the entities from the QUESTION to tables and columns in the DATABASE SCHEMA?
    - AmbOutput: What fields and how many fields should be included in the output table?
    - AmbValue: What predicate value should be used to filter results?
*/

/* The clarification question should be easy to understand for people with no coding experience. */

/* Let's think step by step to generate the helpful multiple choice clarification question.
1. Summarize the clear information based on previous clarification questions and incorrect queries.
2. Evaluate whether AmbQuestion, AmbTableColumn, AmbOutput, and AmbValue remain in formulating an SQL query, considering each category individually.
3. If no remaining ambiguities are identified, then output "NO AMBIGUITY".
   Else, ask a new multiple-choice question to address the remaining ambiguities and assist in identifying the correct SQL query. Use format: mul_choice_cq = "".
*/

"""


In [39]:
# sql_generation = '''Complete sqlite SQL query only and with no explanation.
# Your task is to write a SQLITE SQL QUERY that correctly answers the following QUESTION and align with database schema.

# QUESTION: {question}

# The DATABASE SCHEMA: {schema}

# Answer:'''


# sql_generation_v2 = '''Complete a SQLite SQL query only with no explanation.
# Your task is to complete a new SQLite SQL QUERY that correctly answers the USER QUESTION, aligns with DATABASE SCHEMA, and follows USER FEEDBACK.

# USER QUESTION: ```{question}```
# The DATABASE SCHEMA: ```{schema}```

# The user has provided a list of incorrect SQL queries and the following sql queries are wrong.
# Incorrect Queries: ```[{sqls}]``` 

# The user has provided some feedback. User Feedback are the golden truth.
# User Feedback: {cqas}

# You know the Incorrect Queries are wrong. Do not rewrite the exact same incorrect queries.
# Please listen carefully to the user feedback. Write the new correct query.

# Answer:
# '''

# fix_invalid_v1 = """Your task is to fix the invalid SQL query.

# Database schema```{schema}```

# Invalid SQL Query: ```{sql}```
# The above SQL query is not valid and can not be executed.
# Exception Message:```{ex}```

# Fix the errors and output a new SQL query that is executable and align with Database schema.

# Write the new executable SQL query in the format: sql = ``` ```.

# Answer:
# """

# sql_generation_selfdebug = '''Complete a SQLite SQL query only with no explanation.
# Your task is to complete a new SQLite SQL QUERY that correctly answers the USER QUESTION, aligns with DATABASE SCHEMA, and follows USER FEEDBACK.

# USER QUESTION: ```{question}```
# The DATABASE SCHEMA: ```{schema}```

# Incorrect Queries: ```[{sqls}]```
# The above SQL predictions are wrong. Please fix the SQL.
# Write the new correct query.

# Answer:
# '''


In [46]:
# DAIL SQLNoRule
sql_generation = '''/* Given the following database schema: */
{schema}
/* Answer the following with no explanation: {question} */
SELECT '''

# update to code representation
sql_generation_v2 = '''/* Given the following database schema: */
{schema}
/* And the following incorrect sql answers: */
{sqls}
/* And the following user replies to help you write the correct sql query: */
{cqas}
/* Answer the following with no explanation: {question} */
SELECT '''

# update to code representation
fix_invalid_v1 = """/* Given the following database schema: */
{schema}`
/* And the following inexecutable sql query */
{invalidSQL}
/* And the following exception message */
{ex}

/* Fix the exception and write a new executable SQL query with no explanation */
SELECT """

# update to code representation
sql_generation_selfdebug = '''/* Given the following database schema: */
{schema}
/* And the following incorrect sql answers: */
{sqls}
/* Answer the following with no explanation: {question} */
SELECT '''


In [104]:
def generate_db_schema(database):
    db_path = f'./bird_dev/dev_databases/{database}/{database}.sqlite'
    conn = sqlite3.connect(db_path, uri=True)
    full_schema_prompt_list = []
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    schemas = {}
    for table in tables:
        if table == 'sqlite_sequence':
            continue
        cursor.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name='{}';".format(table[0]))
        create_prompt = cursor.fetchone()[0]
        schemas[table[0]] = create_prompt

    for k, v in schemas.items():
        full_schema_prompt_list.append(v)

    schema_prompt = "\n\n".join(full_schema_prompt_list)

    return schema_prompt

In [149]:
def startBenchBaseline(data_frame, history_log, log_name, rounds, num_of_tests, model_name, data_source):
    assert model_name in ['gpt35turbo', 'gpt4turbo']
    generation = None
    if model_name == 'gpt35turbo':
        generation = GPT35_generation
    else:
        generation = GPT4_turbo_generation
    for index in range(num_of_tests):
        if index in history_log and "num_cq_asked" in history_log[index]:
            # skip tests already seen
            continue
        assert index in history_log
        # we are using batch api to genearte the 0th round
        # and read 0th sql from the log
        assert len(history_log[index]['sql_log']) == 1
        
        cqs_and_answers = []
        query = set()
        d = data_frame.iloc[[index]] 
        if data_source == 'kaggle':
            gold = d['sql'].values[0]
            dbname = d['target_db'].values[0]
            nlq = d['nl'].values[0]
            dbschema = d['target_schema'].values[0]
        elif data_source == 'bird':
            gold = d['SQL'].values[0]
            nlq = d['question'].values[0]
            dbname = d['db_id'].values[0]
            dbschema = generate_db_schema(dbname)
        print("nl: ", nlq, index, dbname)
        print("gold: ", gold)
        
#         sql_prompt = sql_generation.format(schema=dbschema, question=nlq)
#         sql_query, pscore= generation(sql_prompt)
#         history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
        order, sql_prompt, sql_query, pscore = history_log[index]['sql_log'][0]
        order += 1
        sql_query = clean_query(sql_query)
        print("sql: ", sql_query, pscore)
        query.add(sql_query)
        execution, exception = evalfunc(sql_query, gold, dbname, data_source)
        if exception:
            most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
            query.remove(most_recent_sql)
            invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=nlq,\
                                                   invalidSQL=most_recent_sql, ex=exception[0])
            sql, pscore= generation(invalid_prompt)
            sql = clean_query(sql)
            print("After Fix Invalid SQL: ", sql)
            history_log[index]['sql_log'].append((order, invalid_prompt, sql, pscore))
            order += 1
            query.add(sql)
            execution, _ = evalfunc(sql, gold, dbname, data_source)
        if execution:
            history_log[index]['num_cq_asked'] = 0
            print()
            print("-----execution match-----")
            print()
            continue
        for turn in range(rounds):
            sql_prompt = sql_generation_selfdebug.format(schema=dbschema, question=nlq,\
                                              sqls=",\n".join(query))
            sql_query, pscore= generation(sql_prompt)
            sql_query = clean_query(sql_query)
            print("sql: ", sql_query, pscore)
            history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
            order += 1
            sql_query = clean_query(sql_query)
            query.add(sql_query)
            execution, exception = evalfunc(sql_query, gold, dbname, data_source)
            if exception:
                most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
                query.remove(most_recent_sql)
                invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=nlq,\
                                                     invalidSQL=most_recent_sql, ex=exception[0])
                sql, pscore= generation(invalid_prompt)
                sql = clean_query(sql)
                query.add(sql)
                history_log[index]['sql_log'].append((order, invalid_prompt, sql, pscore))
                order += 1
                execution, _ = evalfunc(sql, gold, dbname, data_source)
            if execution:
                history_log[index]['num_cq_asked'] = turn + 1
                print()
                print("********execution match*********")
                print()
                break
        if 'num_cq_asked' not in history_log[index]:
            history_log[index]['num_cq_asked'] = "Failed"
        print('')
        print("------next question------")
        print('')
    save(log_name, history_log)

In [167]:
def askClarificationQuestions(data_frame, history_log, log_name, rounds, num_of_tests, model_name, data_source):
    assert model_name in ['gpt35turbo', 'gpt4turbo']
    generation = None
    if model_name == 'gpt35turbo':
        generation = GPT35_generation
    else:
        generation = GPT4_turbo_generation
    for index in range(num_of_tests):
        if index in history_log and "num_cq_asked" in history_log[index]:
            # skip tests already seen
            continue
        assert index in history_log
        assert len(history_log[index]['sql_log']) == 1, print(index)
        
        d = data_frame.iloc[[index]] 
        cqs_and_answers = []
        query = set()
        if data_source == 'kaggle':
            gold = d['sql'].values[0]
            dbname = d['target_db'].values[0]
            nlq = d['nl'].values[0]
            dbschema = d['target_schema'].values[0]
        elif data_source == 'bird':
            gold = d['SQL'].values[0]
            nlq = d['question'].values[0]
            dbname = d['db_id'].values[0]
            dbschema = generate_db_schema(dbname)
        print("nl: ", nlq, index, dbname)
        print("gold: ", gold)
        
        order, sql_prompt, sql_query, pscore = history_log[index]['sql_log'][0]
        order += 1
        sql_query = clean_query(sql_query)
        print("sql: ", sql_query)
        query.add(sql_query)
        execution, exception = evalfunc(sql_query, gold, dbname, data_source)
        
        if exception:
            most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
            query.remove(most_recent_sql)
            invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=nlq,\
                                                 invalidSQL=most_recent_sql, ex=exception[0])
            sql, pscore= generation(invalid_prompt)
            sql = clean_query(sql)
            history_log[index]['sql_log'].append((order, invalid_prompt, sql, pscore))
            order += 1
            query.add(sql)
            execution, _ = evalfunc(sql, gold, dbname, data_source)
        if execution:
            history_log[index]['num_cq_asked'] = 0
            print()
            print("-----execution match-----")
            print()
            continue
        for turn in range(rounds):
            cqas = ""
            for i in range(len(cqs_and_answers)):
                if i%2 == 0:
                    cqas += "multiple choice clarification question: "+cqs_and_answers[i]+'\n'
                else:
                    cqas += "user: "+cqs_and_answers[i]+'\n'
            if len(cqs_and_answers) == 0:
                cqas = "no previous clarification question.\n"
            cq_prompt = SRA.format(schema=dbschema, question=nlq,\
                                            sqls=",\n".join(query), cqs=cqas)
            cq_prompt = cq_prefix_v1 + cq_prompt
            cq, pscore= generation(cq_prompt)
            history_log[index]['cq_log'].append((order, cq_prompt, cq, pscore))
            order += 1
            print("cq: ", cq)
            if "mul_choice_cq = " in cq:
                cq = cq.split("mul_choice_cq = ")[-1]
            
            feedback_prompt = feedback_v2.format(query = gold, question = cq, nlq=nlq)
            feedback_prompt = feedback_prefix_v1 + feedback_prompt
            feedback, pscore= GPT4o_generation(feedback_prompt)
            print()
            print("feedback, ", feedback)
            history_log[index]['feedback_log'].append((order, feedback_prompt, feedback, pscore))
            order += 1
            if "answer_to_cq =" in feedback:
                feedback = feedback.split("answer_to_cq =")[-1]
            cqs_and_answers.append(cq)
            cqs_and_answers.append(feedback)
        
            # fix incorrect sql based on user feedback
            cqas = ""
            for i in range(len(cqs_and_answers)):
                if i%2 == 0:
                    cqas += "multiple choice clarification question: "+cqs_and_answers[i]+'\n'
                else:
                    cqas += "user: "+cqs_and_answers[i]+'\n'
            if cqas == '':
                cqas = 'no previous clarification questions are asked.\n'
            sql_prompt = sql_generation_v2.format(schema=dbschema, question=nlq,\
                                              sqls="\n".join(query), cqas=cqas)

            sql_query, pscore= generation(sql_prompt)
            sql_query = clean_query(sql_query)
            print("sql: ", sql_query)
            history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
            order += 1
            query.add(sql_query)
            execution, exception = evalfunc(sql_query, gold, dbname, data_source)
            if exception:
                most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
                query.remove(most_recent_sql)
                invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=nlq,\
                                                       invalidSQL=most_recent_sql, ex=exception[0])
                sql, pscore= generation(invalid_prompt)
                sql = clean_query(sql)
                query.add(sql)
                history_log[index]['sql_log'].append((order, invalid_prompt, sql, pscore))
                order += 1
                execution, _ = evalfunc(sql, gold, dbname, data_source)
            if execution:
                history_log[index]['num_cq_asked'] = turn + 1
                print()
                print("********execution match*********")
                print()
                break
                
        if 'num_cq_asked' not in history_log[index]:
            history_log[index]['num_cq_asked'] = "Failed"
        print('')
        print("------next question------")
        print('')
    save(log_name, history_log)

In [ ]:
def askCQsBreakNoAmb(data_frame, history_log, log_name, rounds, num_of_tests, model_name, data_source):
    assert model_name in ['gpt35turbo', 'gpt4turbo']
    generation = None
    if model_name == 'gpt35turbo':
        generation = GPT35_generation
    else:
        generation = GPT4_turbo_generation
    for index in range(num_of_tests):
        if index in history_log and "num_cq_asked" in history_log[index]:
            # skip tests already seen
            continue
            
        assert index in history_log
        assert len(history_log[index]['sql_log']) == 1
        d = data_frame.iloc[[index]] 
        cqs_and_answers = []
        query = set()
        if data_source == 'kaggle':
            gold = d['sql'].values[0]
            dbname = d['target_db'].values[0]
            nlq = d['nl'].values[0]
            dbschema = d['target_schema'].values[0]
        elif data_source == 'bird':
            gold = d['SQL'].values[0]
            nlq = d['question'].values[0]
            dbname = d['db_id'].values[0]
            dbschema = generate_db_schema(dbname)
        print("nl: ", nlq, index, dbname)
        print("gold: ", gold)
        
        order, sql_prompt, sql_query, pscore = history_log[index]['sql_log'][0]
        order += 1
        sql_query = clean_query(sql_query)
        print("sql: ", sql_query, pscore)
        query.add(sql_query)
        execution, exception = evalfunc(sql_query, gold, dbname, data_source)
        
        if exception:
            most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
            query.remove(most_recent_sql)
            invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=nlq,\
                                                 invalidSQL=most_recent_sql, ex=exception[0])
            sql, pscore= generation(invalid_prompt)
            sql = clean_query(sql)
            history_log[index]['sql_log'].append((order, invalid_prompt, sql, pscore))
            order += 1
            query.add(sql)
            execution, _ = evalfunc(sql, gold, dbname, data_source)
        if execution:
            history_log[index]['num_cq_asked'] = 0
            print()
            print("-----execution match-----")
            print()
            continue
        for turn in range(rounds):
            cqas = ""
            for i in range(len(cqs_and_answers)):
                if i%2 == 0:
                    cqas += "Multiple Choice Clarification Question: "+cqs_and_answers[i]
                else:
                    cqas += "Answer: "+cqs_and_answers[i]
            if cqas == '':
                cqas = 'no previous clarification questions.\n'
                
            cq_prompt = SRA_ES.format(schema=dbschema, question=nlq,\
                                            sqls=",\n".join(query), cqs=cqas)
            cq_prompt = cq_prefix_v1 + cq_prompt
            cq, pscore= generation(cq_prompt)
            history_log[index]['cq_log'].append((order, cq_prompt, cq, pscore))
            order += 1
#             print("cq: ", cq, pscore)
            if "NO AMBIGUITY" in cq:
                print()
                print("-----NO AMBGUITY-----")
                print()
                break
            if "mul_choice_cq = " in cq:
                cq = cq.split("mul_choice_cq = ")[-1]
            
            feedback_prompt = feedback_v2.format(query = gold, question = cq, nlq=nlq)
            feedback_prompt = feedback_prefix_v1 + feedback_prompt
            feedback, pscore= GPT4o_generation(feedback_prompt)
#             print()
#             print("feedback, ", feedback, pscore)
            history_log[index]['feedback_log'].append((order, feedback_prompt, feedback, pscore))
            order += 1
            if "answer_to_cq =" in feedback:
                feedback = feedback.split("answer_to_cq =")[-1]
            cqs_and_answers.append(cq)
            cqs_and_answers.append(feedback)
        
            # fix incorrect sql based on user feedback
            cqas = ""
            for i in range(len(cqs_and_answers)):
                if i%2 == 0:
                    cqas += "Multiple Choice Clarification Question: "+cqs_and_answers[i]
                else:
                    cqas += "User Feedback: "+cqs_and_answers[i] 
            if cqas == '':
                cqas = 'no previous clarification questions.\n'
            sql_prompt = sql_generation_v2.format(schema=dbschema, question=nlq,\
                                              sqls="\n".join(query), cqas=cqas)

            sql_query, pscore= generation(sql_prompt)
            print("sql: ", sql_query, pscore)
            history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
            order += 1
            sql_query = clean_query(sql_query)
            query.add(sql_query)
            execution, exception = evalfunc(sql_query, gold, dbname, data_source)
            if exception:
                most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
                query.remove(most_recent_sql)
                invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=nlq,\
                                                       invalidSQL=most_recent_sql, ex=exception[0])
                sql, pscore= generation(invalid_prompt)
                sql = clean_query(sql)
                query.add(sql)
                history_log[index]['sql_log'].append((order, invalid_prompt, sql, pscore))
                order += 1
                execution, _ = evalfunc(sql, gold, dbname, data_source)
            if execution:
                history_log[index]['num_cq_asked'] = turn + 1
                print()
                print("********execution match*********")
                print()
                break
                
        if 'num_cq_asked' not in history_log[index]:
            history_log[index]['num_cq_asked'] = "Failed"
        print('')
        print("------next question------")
        print('')
    save(log_name, history_log)